In [1]:
import pandas as pd
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder

import sys
import os

ruta_carpeta = '/Users/raul_/Desktop/PBREAK/Funciones/'
sys.path.append(ruta_carpeta) 
from Funciones import encuentros_entre_equipos
from Funciones import filter_columns_by_correlation
from Funciones import remove_accents
from Funciones import determinar_resultado
from pandas.plotting import scatter_matrix
from Funciones import *
import matplotlib.pyplot as plt




#### CARGA DEL CSV PARA CONVERTIRLO EN DATAFRAME

In [2]:
resultados = pd.read_csv('Temporada2324.csv', encoding='latin1')

In [3]:
equipos_duplicados = resultados.loc[resultados["Unnamed: 3"] == "@"]
resultados = resultados.drop(equipos_duplicados.index)

In [4]:
resultados.drop(columns=["Unnamed: 3","Poss","G-PK","PK","PKatt","PKm","Match Report","Rk"], inplace=True)

In [5]:
resultados.reset_index(drop=True, inplace=True)

In [6]:
resultados.Date = pd.to_datetime(resultados.Date)

In [7]:
resultados = resultados.map(remove_accents)

In [8]:
resultados.drop_duplicates(subset=resultados.columns, inplace=True)


In [9]:
resultados.rename(columns={"Team" : "Local",
                           "Opp":"Visitante",
                           "Comp" : "Competicion",
                           "GF" : "Goles_Local",
                           "GA" : "Goles_Visitantes",
                           "GD" : "Diferencia_Goles"
                           }, inplace=True)

In [10]:
# Transformar los campos a tipo entero
resultados['Goles_Local'] = resultados['Goles_Local'].astype(int)
resultados['Goles_Visitantes'] = resultados['Goles_Visitantes'].astype(int)
resultados['Diferencia_Goles'] = resultados['Diferencia_Goles'].astype(int)

In [11]:
resultados["Resultado"] = resultados.apply(determinar_resultado, axis=1)
resultados.drop(columns=["Result","Diferencia_Goles"], inplace=True)

In [12]:
orden_columnas = ["Date", "Local", "Goles_Local","Visitante", "Goles_Visitantes","Resultado"]

In [13]:
resultados = resultados[orden_columnas]

In [14]:

resultados['Date'] = pd.to_datetime(resultados['Date'])

# Creeamos Temporada como columna
resultados['Temporada'] = resultados['Date'].apply(lambda x: x.year if x.month >= 8 else x.year - 1)

# Ordenamos por fecha
resultados = resultados.sort_values(by='Date')

# Creamos Jornada
resultados['Jornada'] = resultados.groupby('Temporada').cumcount() // 10 + 1

resultados.reset_index(drop=True, inplace=True)

In [15]:
resultados["Numero_Goles_Total"] = resultados.Goles_Local + resultados.Goles_Visitantes

In [16]:
resultados["Puntos_Local"] = 0
resultados["Puntos_Visitantes"] = 0

In [17]:

for i, row in resultados.iterrows():
    if row['Resultado'] == "Local":
        resultados.at[i, 'Puntos_Local'] += 3
    elif row['Resultado'] == "Visitante":
        resultados.at[i, 'Puntos_Visitantes'] += 3
    else:
        resultados.at[i, 'Puntos_Local'] += 1
        resultados.at[i, 'Puntos_Visitantes'] += 1

In [18]:

# Supongamos que 'resultados' es tu DataFrame original
# Crear DataFrames separados para los equipos locales y visitantes
df_local = resultados[['Temporada', 'Jornada', 'Local', 'Puntos_Local']].rename(columns={'Local': 'Equipo', 'Puntos_Local': 'Puntos'})
df_visitante = resultados[['Temporada', 'Jornada', 'Visitante', 'Puntos_Visitantes']].rename(columns={'Visitante': 'Equipo', 'Puntos_Visitantes': 'Puntos'})

# Unir ambos DataFrames
df_puntos = pd.concat([df_local, df_visitante])

# Ordenar por temporada, equipo y jornada
df_puntos = df_puntos.sort_values(by=['Temporada', 'Equipo', 'Jornada'])

# Calcular la suma acumulada de puntos para cada equipo en cada temporada, excluyendo la jornada actual
df_puntos['Puntos_Acumulados'] = df_puntos.groupby(['Temporada', 'Equipo'])['Puntos'].shift().fillna(0).groupby(df_puntos['Temporada']).cumsum()



In [19]:

# Supongamos que 'resultados' es tu DataFrame original
# Crear DataFrames separados para los equipos locales y visitantes
df_local = resultados[['Temporada', 'Jornada', 'Local', 'Puntos_Local']].rename(columns={'Local': 'Equipo', 'Puntos_Local': 'Puntos'})
df_visitante = resultados[['Temporada', 'Jornada', 'Visitante', 'Puntos_Visitantes']].rename(columns={'Visitante': 'Equipo', 'Puntos_Visitantes': 'Puntos'})

# Unir ambos DataFrames
df_puntos = pd.concat([df_local, df_visitante])

# Ordenar por temporada, equipo y jornada
df_puntos = df_puntos.sort_values(by=['Temporada', 'Equipo', 'Jornada'])

# Calcular la suma acumulada de puntos para cada equipo en cada temporada, excluyendo la jornada actual
df_puntos['Puntos_Acumulados'] = df_puntos.groupby(['Temporada', 'Equipo'])['Puntos'].shift().fillna(0).groupby([df_puntos['Temporada'], df_puntos['Equipo']]).cumsum()

# Crear un DataFrame para los puntos acumulados de los equipos locales
df_puntos_local = df_puntos.rename(columns={'Equipo': 'Local', 'Puntos_Acumulados': 'Puntos_Acumulados_Local'})

# Combinar los puntos acumulados locales con el DataFrame original
resultados = resultados.merge(df_puntos_local[['Temporada', 'Jornada', 'Local', 'Puntos_Acumulados_Local']],
                              on=['Temporada', 'Jornada', 'Local'], how='left')

# Crear un DataFrame para los puntos acumulados de los equipos visitantes
df_puntos_visitante = df_puntos.rename(columns={'Equipo': 'Visitante', 'Puntos_Acumulados': 'Puntos_Acumulados_Visitantes'})

# Combinar los puntos acumulados visitantes con el DataFrame original
resultados = resultados.merge(df_puntos_visitante[['Temporada', 'Jornada', 'Visitante', 'Puntos_Acumulados_Visitantes']],
                              on=['Temporada', 'Jornada', 'Visitante'], how='left')




In [20]:
# Crear un DataFrame para almacenar las posiciones en la liga
df_posiciones = pd.DataFrame()

# Iterar sobre cada temporada
for season in df_puntos['Temporada'].unique():
    
    df_season = df_puntos[df_puntos['Temporada'] == season]
    
    # Iterar sobre cada jornada
    for jornada in df_season['Jornada'].unique():
        
        df_jornada = df_season[df_season['Jornada'] == jornada]
        
        df_jornada = df_jornada.sort_values(by='Puntos_Acumulados', ascending=False).reset_index(drop=True)
        
        df_jornada['Posicion'] = df_jornada.index + 1
        
        # Agregar al DataFrame de posiciones
        df_posiciones = pd.concat([df_posiciones, df_jornada[['Temporada', 'Jornada', 'Equipo', 'Puntos_Acumulados', 'Posicion']]])



In [21]:
# Eliminar duplicados en todo el conjunto de datos manteniendo una sola entrada por partido
resultados = resultados.drop_duplicates(subset=['Date', 'Local', 'Visitante'])

In [22]:
# Ahora vamos a combinar las posiciones calculadas con el DataFrame original
# Primero, renombrar columnas en df_posiciones para facilitar el merge
df_posiciones_local = df_posiciones.rename(columns={'Equipo': 'Local', 'Posicion': 'Posicion_Local'})
df_posiciones_visitante = df_posiciones.rename(columns={'Equipo': 'Visitante', 'Posicion': 'Posicion_Visitante'})

# Combinar las posiciones de los equipos locales con el DataFrame original
resultados = resultados.merge(df_posiciones_local[['Temporada', 'Jornada', 'Local', 'Posicion_Local']],
                                                             on=['Temporada', 'Jornada', 'Local'], how='left')

# Combinar las posiciones de los equipos visitantes con el DataFrame original
resultados = resultados.merge(df_posiciones_visitante[['Temporada', 'Jornada', 'Visitante', 'Posicion_Visitante']],
                                                                on=['Temporada', 'Jornada', 'Visitante'], how='left')



In [23]:


# Supongamos que 'resultados' es tu DataFrame original
# Crear DataFrames separados para los equipos locales y visitantes
df_local = resultados[['Temporada', 'Jornada', 'Local', 'Goles_Local']].rename(columns={'Local': 'Equipo', 'Goles_Local': 'Goles'})
df_visitante = resultados[['Temporada', 'Jornada', 'Visitante', 'Goles_Visitantes']].rename(columns={'Visitante': 'Equipo', 'Goles_Visitantes': 'Goles'})

# Unir ambos DataFrames
df_goles = pd.concat([df_local, df_visitante])

# Ordenar por temporada, equipo y jornada
df_goles = df_goles.sort_values(by=['Temporada', 'Equipo', 'Jornada'])

# Calcular la suma acumulada de goles para cada equipo en cada temporada, excluyendo la jornada actual
df_goles['Goles_Acumulados'] = df_goles.groupby(['Temporada', 'Equipo'])['Goles'].shift().fillna(0).groupby([df_goles['Temporada'], df_goles['Equipo']]).cumsum()

# Crear un DataFrame para los goles acumulados de los equipos locales
df_goles_local = df_goles.rename(columns={'Equipo': 'Local', 'Goles_Acumulados': 'Goles_Acumulados_Local'})

# Combinar los goles acumulados locales con el DataFrame original
resultados = resultados.merge(df_goles_local[['Temporada', 'Jornada', 'Local', 'Goles_Acumulados_Local']],
                              on=['Temporada', 'Jornada', 'Local'], how='left')

# Crear un DataFrame para los goles acumulados de los equipos visitantes
df_goles_visitante = df_goles.rename(columns={'Equipo': 'Visitante', 'Goles_Acumulados': 'Goles_Acumulados_Visitantes'})

# Combinar los goles acumulados visitantes con el DataFrame original
resultados = resultados.merge(df_goles_visitante[['Temporada', 'Jornada', 'Visitante', 'Goles_Acumulados_Visitantes']],
                              on=['Temporada', 'Jornada', 'Visitante'], how='left')



In [24]:
# Eliminar duplicados en todo el conjunto de datos manteniendo una sola entrada por partido
resultados = resultados.drop_duplicates(subset=['Date', 'Local', 'Visitante'])

In [25]:
resultados.drop(columns=["Puntos_Local","Puntos_Visitantes"], inplace=True)

In [26]:
nuevo_orden = ["Date","Temporada","Jornada","Posicion_Local","Puntos_Acumulados_Local","Local","Goles_Local","Goles_Acumulados_Local","Visitante","Goles_Visitantes","Goles_Acumulados_Visitantes","Puntos_Acumulados_Visitantes","Posicion_Visitante","Resultado","Numero_Goles_Total"]

In [27]:
resultados = resultados[resultados['Resultado'] != 'Empate']

In [28]:
resultados = resultados[nuevo_orden]

In [29]:

encoded = resultados.copy()


label_encoder = LabelEncoder()
categorical_columns = ['Date', 'Local', 'Visitante', 'Resultado']

for column in categorical_columns:
   encoded[column] = label_encoder.fit_transform(encoded[column])

In [30]:
resultados["Ratio_Jornada_Local"] = resultados.Puntos_Acumulados_Local / resultados.Jornada
resultados["Ratio_Jornada_Visitante"] = resultados.Puntos_Acumulados_Visitantes / resultados.Jornada

In [31]:
resultados = clasificacion(resultados)

In [32]:
resultados.drop_duplicates(subset=resultados.columns, inplace=True)


In [33]:
resultados.drop_duplicates(subset=resultados.columns, inplace=True)

In [34]:
# Crear columnas para el cálculo de forma
resultados['Resultado_Local'] = resultados.apply(lambda x: calcular_resultado(x, x['Local']), axis=1)  #Generamos dos columnas nuevas para almacenar el resultado local y el visitante.
resultados['Resultado_Visitante'] = resultados.apply(lambda x: calcular_resultado(x, x['Visitante']), axis=1)

resultados['Racha_Local'] = resultados.apply(lambda x: actualizar_forma(forma_local, x['Local'], x['Resultado_Local']), axis=1)
resultados['Racha_Visitante'] = resultados.apply(lambda x: actualizar_forma(forma_visitante, x['Visitante'], x['Resultado_Visitante']), axis=1)

In [35]:

resultados['Racha_Puntos_Local'] = resultados['Racha_Local'].apply(clasificador_rendimiento_forma)
resultados['Racha_Puntos_Visitante'] = resultados['Racha_Visitante'].apply(clasificador_rendimiento_forma)

In [36]:
# Eliminar duplicados en todo el conjunto de datos manteniendo una sola entrada por partido
resultados = resultados.drop_duplicates(subset=['Date', 'Local', 'Visitante'])

In [37]:
resultados.reset_index(drop=True, inplace=True)


In [38]:

# Crear un identificador único para cada enfrentamiento
resultados['Match_ID'] = resultados.apply(lambda x: '_'.join(sorted([x['Local'], x['Visitante']])), axis=1)

# Ordenar el dataframe por fecha para mantener el orden cronológico
df = resultados.sort_values(by='Date')

# Crear un dataframe para almacenar los resultados
result_df = pd.DataFrame()

# Usar groupby para agrupar por el identificador de enfrentamiento
grouped = df.groupby('Match_ID')

# Procesar cada grupo
for match_id, group in grouped:
    group = group.sort_values(by='Date')
    group['Cumulative_Goles_Local'] = group['Goles_Local'].cumsum().shift(1)
    group['Cumulative_Goles_Visitante'] = group['Goles_Visitantes'].cumsum().shift(1)
    group['Partidos_Jugados'] = range(1, len(group) + 1)
    group['Cumulative_Goles_Total'] = group['Cumulative_Goles_Local'] + group['Cumulative_Goles_Visitante']
    group['Media_Goles'] = group['Cumulative_Goles_Total'] / (group['Partidos_Jugados'] - 1)
    result_df = pd.concat([result_df, group])

# Limpiar las columnas temporales
result_df = result_df[['Date', 'Temporada', 'Local', 'Visitante', 'Media_Goles']]





In [39]:
# Sustituir NaN por 0 en la columna 'Media_Goles'
result_df['Media_Goles'] = result_df['Media_Goles'].fillna(0)

In [40]:
# Unir la columna 'Media_Goles' con el dataframe original 'df' basado en las columnas 'Date', 'Temporada', 'Local' y 'Visitante'
resultados = resultados.merge(result_df[['Date', 'Temporada', 'Local', 'Visitante', 'Media_Goles']],
                     on=['Date', 'Temporada', 'Local', 'Visitante'], 
                     how='left')


In [41]:
# Crear un identificador único para cada enfrentamiento
resultados['Match_ID'] = resultados.apply(lambda x: '_'.join(sorted([x['Local'], x['Visitante']])), axis=1)

# Ordenar el dataframe por fecha para mantener el orden cronológico
df = df.sort_values(by='Date')

# Crear un dataframe para almacenar los resultados
result_df_individual = pd.DataFrame()

# Usar groupby para agrupar por el identificador de enfrentamiento
grouped = df.groupby('Match_ID')

# Procesar cada grupo
for match_id, group in grouped:
    group = group.sort_values(by='Date')
    group['Cumulative_Goles_Local'] = group.apply(lambda x: group[(group['Local'] == x['Local']) & (group['Date'] < x['Date'])]['Goles_Local'].sum(), axis=1)
    group['Cumulative_Goles_Visitante'] = group.apply(lambda x: group[(group['Visitante'] == x['Visitante']) & (group['Date'] < x['Date'])]['Goles_Visitantes'].sum(), axis=1)
    group['Partidos_Jugados_Local'] = group.apply(lambda x: len(group[(group['Local'] == x['Local']) & (group['Date'] < x['Date'])]), axis=1)
    group['Partidos_Jugados_Visitante'] = group.apply(lambda x: len(group[(group['Visitante'] == x['Visitante']) & (group['Date'] < x['Date'])]), axis=1)
    group['Media_Goles_Local'] = group['Cumulative_Goles_Local'] / group['Partidos_Jugados_Local'].replace(0, 1)
    group['Media_Goles_Visitante'] = group['Cumulative_Goles_Visitante'] / group['Partidos_Jugados_Visitante'].replace(0, 1)
    result_df_individual = pd.concat([result_df_individual, group])

# Limpiar las columnas temporales
result_df_individual = result_df_individual[['Date', 'Temporada', 'Local', 'Visitante', 'Media_Goles_Local', 'Media_Goles_Visitante']]

# Unir los resultados individuales con el dataframe original
resultados = resultados.merge(result_df_individual[['Date', 'Temporada', 'Local', 'Visitante', 'Media_Goles_Local', 'Media_Goles_Visitante']],
                                on=['Date', 'Temporada', 'Local', 'Visitante'], 
                                how='left')


In [42]:


# Crear un diccionario para asignar puntos según la posición (20 puntos para el 1ro, 1 punto para el 20mo)
points_dict = {i: 21-i for i in range(1, 21)}

# Filtrar las posiciones finales de la jornada 38 para equipos locales
df_positions_local = resultados[resultados['Jornada'] == 38][['Temporada', 'Local', 'Posicion_Local']].drop_duplicates()
df_positions_local['Puntos'] = df_positions_local['Posicion_Local'].map(points_dict)
df_positions_local.columns = ['Temporada', 'Equipo', 'Posicion', 'Puntos']

# Filtrar las posiciones finales de la jornada 38 para equipos visitantes
df_positions_visitante = resultados[resultados['Jornada'] == 38][['Temporada', 'Visitante', 'Posicion_Local']].drop_duplicates()
df_positions_visitante.columns = ['Temporada', 'Equipo', 'Posicion']
df_positions_visitante['Puntos'] = df_positions_visitante['Posicion'].map(points_dict)

# Concatenar ambos dataframes
df_positions = pd.concat([df_positions_local, df_positions_visitante])

# Sumar los puntos por cada equipo a lo largo de todas las temporadas
ranking_df = df_positions.groupby('Equipo')['Puntos'].sum().reset_index()
ranking_df.columns = ['Equipo', 'Puntos_Totales']

# Unir los resultados del ranking con el dataframe original basado en el equipo local
resultados = resultados.merge(ranking_df, left_on='Local', right_on='Equipo', how='left')
resultados.rename(columns={'Puntos_Totales': 'Ranking_Local'}, inplace=True)
resultados.drop(columns=['Equipo'], inplace=True)

# Unir los resultados del ranking con el dataframe original basado en el equipo visitante
resultados = resultados.merge(ranking_df, left_on='Visitante', right_on='Equipo', how='left')
resultados.rename(columns={'Puntos_Totales': 'Ranking_Visitante'}, inplace=True)
resultados.drop(columns=['Equipo'], inplace=True)


In [43]:

# Ordenar el DataFrame por temporada, jornada y fecha para asegurarnos de que los cálculos son correctos
resultados = resultados.sort_values(by=['Temporada', 'Jornada', 'Date']).reset_index(drop=True)

# Inicializar las columnas de goles encajados y goles marcados
resultados['Goles_Encajados_Local_Acumulados'] = 0
resultados['Goles_Encajados_Visitante_Acumulados'] = 0
resultados['Goles_Marcados_Local_Acumulados'] = 0
resultados['Goles_Marcados_Visitante_Acumulados'] = 0

# Calcular los goles encajados y marcados por el equipo local hasta la jornada actual (sin incluir la jornada actual)
for temporada in resultados['Temporada'].unique():
    for equipo in resultados['Local'].unique():
        mask_local = (resultados['Temporada'] == temporada) & (resultados['Local'] == equipo)
        resultados.loc[mask_local, 'Goles_Encajados_Local_Acumulados'] = resultados.loc[mask_local, 'Goles_Visitantes'].shift().cumsum().fillna(0)
        resultados.loc[mask_local, 'Goles_Marcados_Local_Acumulados'] = resultados.loc[mask_local, 'Goles_Local'].shift().cumsum().fillna(0)

# Calcular los goles encajados y marcados por el equipo visitante hasta la jornada actual (sin incluir la jornada actual)
for temporada in resultados['Temporada'].unique():
    for equipo in resultados['Visitante'].unique():
        mask_visitante = (resultados['Temporada'] == temporada) & (resultados['Visitante'] == equipo)
        resultados.loc[mask_visitante, 'Goles_Encajados_Visitante_Acumulados'] = resultados.loc[mask_visitante, 'Goles_Local'].shift().cumsum().fillna(0)
        resultados.loc[mask_visitante, 'Goles_Marcados_Visitante_Acumulados'] = resultados.loc[mask_visitante, 'Goles_Visitantes'].shift().cumsum().fillna(0)



In [44]:
resultados['Diferencia_Posicion'] = resultados['Posicion_Local'] - resultados['Posicion_Visitante']

In [45]:

# Inicializar columnas de rendimiento y forma reciente
resultados['Rendimiento_Casa_Local'] = 0
resultados['Rendimiento_Fuera_Visitante'] = 0
resultados['Forma_Reciente_Local'] = 0
resultados['Forma_Reciente_Visitante'] = 0

# Calcular rendimiento y forma reciente para cada equipo
teams = resultados['Local'].unique()

# Promedios acumulados y formas recientes
for team in teams:
    # Datos del equipo local y visitante
    local_games = resultados[resultados['Local'] == team]
    visitante_games = resultados[resultados['Visitante'] == team]
    
    # Rendimiento en casa
    local_points = local_games['Puntos_Acumulados_Local'].diff().fillna(local_games['Puntos_Acumulados_Local'])
    resultados.loc[resultados['Local'] == team, 'Rendimiento_Casa_Local'] = local_points.expanding().mean()
    
    # Rendimiento fuera
    visitante_points = visitante_games['Puntos_Acumulados_Visitantes'].diff().fillna(visitante_games['Puntos_Acumulados_Visitantes'])
    resultados.loc[resultados['Visitante'] == team, 'Rendimiento_Fuera_Visitante'] = visitante_points.expanding().mean()
    
    # Forma reciente en casa
    resultados.loc[resultados['Local'] == team, 'Forma_Reciente_Local'] = local_points.rolling(window=3).mean().fillna(local_points.expanding().mean())
    
    # Forma reciente fuera
    resultados.loc[resultados['Visitante'] == team, 'Forma_Reciente_Visitante'] = visitante_points.rolling(window=3).mean().fillna(visitante_points.expanding().mean())



C:\Users\raul_\AppData\Local\Temp\ipykernel_22056\1116965769.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         0.         0.         0.75       2.6        2.66666667
 2.28571429 2.5        2.66666667 2.8        3.09090909 3.16666667
 3.15384615 2.92857143]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc[resultados['Local'] == team, 'Rendimiento_Casa_Local'] = local_points.expanding().mean()
C:\Users\raul_\AppData\Local\Temp\ipykernel_22056\1116965769.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         3.5        4.         3.25       2.6        2.16666667
 2.28571429 2.125      2.66666667 2.8        2.81818182 3.41666667
 3.15384615]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resultados.loc[re

In [ ]:

# Revisar los partidos para saber si el equipo local o visitante recibieron goles basándose en las columnas Goles_Local y Goles_Visitante
resultados['Local_Porteria_Cero'] = resultados['Goles_Visitantes'] == 0
resultados['Visitante_Porteria_Cero'] = resultados['Goles_Local'] == 0

# Crear columnas acumulativas para porterías a 0 por temporada
resultados['Porterias_Cero_Local_Acum'] = resultados.groupby('Temporada')['Local_Porteria_Cero'].cumsum()
resultados['Porterias_Cero_Visitante_Acum'] = resultados.groupby('Temporada')['Visitante_Porteria_Cero'].cumsum()


In [46]:
resultados.drop(columns=["Resultado_Local","Resultado_Visitante","Goles_Visitantes","Goles_Local","Numero_Goles_Total","Match_ID","Local_Porteria_Cero","Visitante_Porteria_Cero","Date"],inplace=True)

KeyError: "['Local_Porteria_Cero', 'Visitante_Porteria_Cero'] not found in axis"

In [ ]:
categorical_columns = resultados.select_dtypes(include=['object']).columns


encoded = resultados.copy()


label_encoder = LabelEncoder()


for column in categorical_columns:
   encoded[column] = label_encoder.fit_transform(encoded[column])

In [ ]:

encoded["Diferencia_Puntos_Local"]= encoded["Puntos_Acumulados_Local"] - encoded["Puntos_Acumulados_Visitantes"] #Diferencia de puntos Local
encoded["Diferencia_Puntos_Visitante"]= encoded["Puntos_Acumulados_Visitantes"] - encoded["Puntos_Acumulados_Local"] #Diferencia de puntos Visitante

encoded["Ratio_Goles_por_partido_Visitante"]= encoded["Goles_Acumulados_Visitantes"] / encoded["Jornada"] #Media de goles Visitante por partido
encoded["Ratio_Goles_por_partido_Local"]= encoded["Goles_Acumulados_Local"] / encoded["Jornada"] #Media de goles Local por partido

encoded["Rendimiento_Ranking_Visitante"]= (encoded["Ranking_Visitante"] * encoded["Ratio_Jornada_Visitante"]) 
encoded["Rendimiento_Ranking_Local"]= (encoded["Ranking_Local"] * encoded["Ratio_Jornada_Local"]) 


In [ ]:
encoded['Diferencia_Ranking'] = encoded['Ranking_Local'] - encoded['Ranking_Visitante']

# Crear nuevas columnas basadas en indicadores binarios mejorados
encoded['Local_Es_Favorito'] = (encoded['Ranking_Local'] < encoded['Ranking_Visitante']).astype(int)
encoded['Visitante_Es_Favorito'] = (encoded['Ranking_Visitante'] < encoded['Ranking_Local']).astype(int)

In [ ]:

# Crear nuevas columnas basadas en interacciones cuadráticas y no lineales
encoded['Ranking_Local_Cuadrado'] = encoded['Ranking_Local'] ** 2
encoded['Ranking_Visitante_Cuadrado'] = encoded['Ranking_Visitante'] ** 2

encoded['Rendimiento_Vistante'] = encoded['Ratio_Goles_por_partido_Visitante'] / encoded['Posicion_Visitante']
encoded['Rendimiento_Local'] = encoded['Ratio_Goles_por_partido_Local'] / encoded['Posicion_Local']


encoded['Local_Es_Ofensivo'] = (encoded['Media_Goles_Local'] < encoded['Media_Goles_Visitante']).astype(int)
encoded['Visitante_Es_Ofensivo'] = (encoded['Media_Goles_Visitante'] < encoded['Media_Goles_Local']).astype(int)


encoded['Local_Es_Defensivo'] = (encoded['Goles_Encajados_Local_Acumulados'] < encoded['Goles_Encajados_Visitante_Acumulados']).astype(int)
encoded['Visitante_Es_Defensivo'] = (encoded['Goles_Encajados_Visitante_Acumulados'] < encoded['Goles_Encajados_Local_Acumulados']).astype(int)

In [ ]:
encoded.drop(columns=["Resultado"], inplace=True)

In [ ]:
encoded = encoded.dropna(axis=1, how='any')

In [ ]:
#Exportamos en CSV
encoded.to_csv("TEST_PRUEBAS_2.csv", index=False)

In [ ]:
encoded.head(2)

,Temporada,Jornada,Posicion_Local,Puntos_Acumulados_Local,Local,Goles_Acumulados_Local,Visitante,Goles_Acumulados_Visitantes,Puntos_Acumulados_Visitantes,Posicion_Visitante,...,Local_Es_Favorito,Visitante_Es_Favorito,Ranking_Local_Cuadrado,Ranking_Visitante_Cuadrado,Rendimiento_Vistante,Rendimiento_Local,Local_Es_Ofensivo,Visitante_Es_Ofensivo,Local_Es_Defensivo,Visitante_Es_Defensivo
0,2023,1,4,0.0,17,0.0,18,0.0,0.0,3,...,0,1,64,49,0.0,0.0,0,0,0,0
1,2023,1,2,0.0,1,0.0,14,0.0,0.0,7,...,1,0,1,25,0.0,0.0,0,0,0,0
